<a href="https://colab.research.google.com/github/Shruti022/Healthcare-Chatbot/blob/main/Copy_of_LLM_Project_pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Phase 1: Stepwise API Exploration

Step 1: Import Libraries


In [125]:
!pip install -q requests pandas streamlit pyngrok faiss-cpu sentence-transformers numpy

import requests
import pandas as pd
import json
import hashlib
from datetime import datetime
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

In [126]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [127]:
# Secure KEY INPUT
import os
import getpass

# Securely Capture Key
# Input will be invisible. Paste key and press Enter.
key_input = getpass.getpass("🔑 Enter Gemini API Key (Invisible Input): ")

if not key_input.startswith("AIza"):
    print("⚠️ Warning: Key might be invalid (usually starts with 'AIza').")
else:
    print("✅ API Key captured securely in Environment Variable.")

# 2. Set as Environment Variable for the Session
os.environ["GEMINI_API_KEY"] = key_input

🔑 Enter Gemini API Key (Invisible Input): ··········
✅ API Key captured securely in Environment Variable.


In [128]:
%%writefile build_embeddings.py
import pandas as pd
import numpy as np
import faiss
import json
from sentence_transformers import SentenceTransformer

# === REAL PATH (from readlink) ===
BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"

# ---------------------------------------------
# Load Data
# ---------------------------------------------
df = pd.read_csv(f"{BASE}/clinical_trials_diabetes_full.csv")

df["status"] = df["status"].astype(str).str.strip().str.title()
bad_status = ["Terminated", "Withdrawn", "Suspended", "No Longer Available", "Unknown"]
df_clean = df[~df["status"].isin(bad_status)].copy()

# ---------------------------------------------
# Chunking
# ---------------------------------------------
chunks = []
chunk_map = []

for idx, row in df_clean.iterrows():
    title = str(row.get("brief_title", "")).strip()
    summary = str(row.get("brief_summary", "")).strip()

    if len(summary) < 20:
        continue

    text = f"Title: {title}\nSummary: {summary}"
    chunks.append(text)

    chunk_map.append({
        "nct_id": row["nct_id"],
        "title": title,
        "text": text,
        "status": row["status"]
    })

print(f"Created {len(chunks)} chunks.")

# ---------------------------------------------
# Embeddings
# ---------------------------------------------
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks, batch_size=64, show_progress_bar=True)

np.save(f"{BASE}/clinical_trials_diabetes_full_embeddings.npy", embeddings)
print("Saved clinical_trials_diabetes_full_embeddings.npy")

# ---------------------------------------------
# Save chunk map
# ---------------------------------------------
with open(f"{BASE}/clinical_trials_diabetes_full_chunk_map.json", "w") as f:
    json.dump(chunk_map, f)

print("Saved clinical_trials_diabetes_full_chunk_map.json")

# ---------------------------------------------
# Build & Save FAISS
# ---------------------------------------------
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))
faiss.write_index(index, f"{BASE}/clinical_trials_diabetes_full_faiss.index")

print("Saved clinical_trials_diabetes_full_faiss.index")
print("✅ Embedding build COMPLETE.")


Overwriting build_embeddings.py


In [129]:
!python build_embeddings.py

2025-11-27 04:03:00.945976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764216180.977338   58701 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764216180.989850   58701 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764216181.006639   58701 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764216181.006665   58701 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764216181.006669   58701 computation_placer.cc:177] computation placer alr

In [130]:
%%writefile utils.py
import json
import hashlib
from datetime import datetime

import faiss
from sentence_transformers import SentenceTransformer

# --- Confidence score from distance ---

def calculate_confidence_score(distance: float, normalization_factor: float = 1.0) -> float:
    """Inverse L2 distance score in (0,1]; closer = higher confidence."""
    return normalization_factor / (normalization_factor + float(distance))


# --- Load pre-built index + chunk map ---

def load_data_and_index(chunk_map_path: str, faiss_path: str):
    """Loads pre-built chunks and FAISS index for quick startup."""
    print("⏳ Loading pre-built RAG index...")

    with open(chunk_map_path, "r") as f:
        chunk_map = json.load(f)

    index = faiss.read_index(faiss_path)

    embed_model = SentenceTransformer("all-MiniLM-L6-v2")

    print(f"✅ RAG Index Ready: {index.ntotal} vectors loaded.")
    return embed_model, index, chunk_map


# --- Provenance logging ---

def log_provenance_step(agent_name: str, input_data, output_data, detail=None):
    """
    Creates a detailed log entry for a single agent step.
    """
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "agent": agent_name,
        "input": input_data,
        "output": output_data,
        "detail": detail or {},
        "model_version": "gemini-2.0-flash",
    }
    return log_entry


# --- Reproducibility hash ---

def generate_reproducibility_hash(conversation_history, corpus_version: str = "v1.0"):
    """
    Generates a deterministic session hash based on the conversation history.
    """
    queries = [turn.get("query", "") for turn in conversation_history]
    raw = f"{corpus_version}|{'|'.join(queries)}"
    return hashlib.md5(raw.encode("utf-8")).hexdigest()


Overwriting utils.py


In [131]:
%%writefile run_bot.py
import json
import re
import os
import sys
from typing import List, Dict, Any
import numpy as np
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

from utils import (
    load_data_and_index,
    log_provenance_step,
    generate_reproducibility_hash,
    calculate_confidence_score,
)

# ==============================
# CONFIG
# ==============================

API_KEY = os.environ.get("GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("❌ GEMINI_API_KEY is not set.")

genai.configure(api_key=API_KEY)
# Using the Experimental 2.0 Flash endpoint
gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")

CHUNK_PATH = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data/clinical_trials_diabetes_full_chunk_map.json"
FAISS_PATH = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data/clinical_trials_diabetes_full_faiss.index"

embed_model, faiss_index, chunk_map = load_data_and_index(CHUNK_PATH, FAISS_PATH)


# ==============================
# TONE MANAGER
# ==============================
class ToneManager:
    tone = "professional"

    @classmethod
    def set_tone(cls, tone: str):
        cls.tone = tone


# ==============================
# PARSER
# ==============================
class QueryParser:
    def __init__(self, model):
        self.model = model

    def parse(self, text):
        parsed = {
            "intent": "trial_search",
            "query": text,
            "is_diabetes_related": True
        }
        return parsed, log_provenance_step("QueryParser", text, parsed)


# ==============================
# RETRIEVAL AGENT
# ==============================
class RetrievalAgent:
    def __init__(self, embed_model, index, chunk_map):
        self.embed_model = embed_model
        self.index = index
        self.chunk_map = chunk_map

    def retrieve(self, parsed, top_k=5):
        q = parsed["query"]
        emb = self.embed_model.encode([q])
        D, I = self.index.search(emb.astype("float32"), top_k)

        trials = []
        for r, idx in enumerate(I[0]):
            item = self.chunk_map[idx]
            conf = calculate_confidence_score(D[0][r])
            trials.append({
                "nct_id": item["nct_id"],
                "title": item["title"],
                "text": item["text"],
                "status": item["status"],
                "confidence": conf,
            })

        return {"query": q, "trials": trials}, log_provenance_step("RetrievalAgent", parsed, trials)


# ==============================
# SUMMARY GENERATOR (No textwrap)
# ==============================
class TrialSummarizer:
    def __init__(self, model):
        self.model = model

    def summarize_trial(self, trial, tone):

        prompt = f"""
You are summarizing a diabetes clinical trial for healthcare relevance.

NCT: {trial['nct_id']}
Title: {trial['title']}
Status: {trial['status']}
Registry Summary:
{trial['text']}

Write:
1️⃣ Abstract — rewrite in {tone} tone, 2–3 sentences
2️⃣ Key Findings — infer expected results based on trial goal (NO invented numbers)
3️⃣ Takeaway — single clinically relevant sentence

No recommendations for medication changes.
Be accurate, concise, and structured.
"""

        response = self.model.generate_content(prompt)
        return response.text.strip()

    def summarize(self, trials):
        return "\n\n---\n\n".join(
            self.summarize_trial(t, ToneManager.tone) for t in trials
        )


# ==============================
# SAFETY FILTER
# ==============================
class SafetyFilter:
    def verify(self, text):
        banned = ["stop taking", "prescribe", "diagnose"]
        if any(x in text.lower() for x in banned):
            return "⚠️ Safety revision: consult your clinician.", "Revised"
        return text, "Pass"


# ==============================
# BOT
# ==============================
class HealthcareBot:
    def __init__(self):
        # Instantiate gemini_model here to ensure it uses the configured API_KEY
        self.gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")
        self.parser = QueryParser(self.gemini_model)
        self.retriever = RetrievalAgent(embed_model, faiss_index, chunk_map)
        self.summarizer = TrialSummarizer(self.gemini_model)
        self.safety = SafetyFilter()

    def process_query(self, user_input):

        if user_input.lower() in ["hi", "hello", "hey"]:
            return {
                "recommendation": (
                    "👋 Hi! I find and summarize **real diabetes clinical trials**.\n\n"
                    "**Try asking:**\n"
                    "• GLP-1 trials for type 2 diabetes\n"
                    "• diet studies for weight loss in diabetes\n"
                    "• insulin pump trials for adults\n"
                )
            }

        parsed, _ = self.parser.parse(user_input)
        retrieved, _ = self.retriever.retrieve(parsed)

        if not retrieved["trials"]:
            return {"recommendation": "No matching trials found."}

        summary_text = self.summarizer.summarize(retrieved["trials"])
        final_text, _ = self.safety.verify(summary_text)

        return {"recommendation": final_text}


_bot = HealthcareBot()

def run_bot(user_input):
    return _bot.process_query(user_input)


Overwriting run_bot.py


UI frontend application simple web interface

https://docs.streamlit.io/develop/tutorials/chat-and-llm-apps/build-conversational-apps

In [132]:
%%writefile app.py
import streamlit as st
from run_bot import run_bot, ToneManager

st.title("Diabetes Clinical Trial Assistant 🔬")

# Tone selector
mode = st.radio("Audience:", ["Professional", "Patient"], index=0)
ToneManager.set_tone(mode.lower())

if "messages" not in st.session_state:
    st.session_state.messages = []

# Chat history
for m in st.session_state.messages:
    with st.chat_message(m["role"]):
        st.markdown(m["content"])

user_input = st.chat_input("Ask about diabetes clinical trials...")
if user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})

    with st.chat_message("user"):
        st.markdown(user_input)

    result = run_bot(user_input)
    reply = result["recommendation"]

    with st.chat_message("assistant"):
        st.markdown(reply)

    st.session_state.messages.append({"role": "assistant", "content": reply})


Overwriting app.py


In [133]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared
!chmod +x cloudflared

In [134]:
#AI LLM
!streamlit run app.py &>/dev/null&
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-11-27T04:03:47Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-27T04:03:47Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-27T04:03:50Z INF +--------------------------------------------------------------------------------------------+
2025-11-27T04:03:50Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-27T04:03:50Z INF |  https://brave-pan-thereof-clara.trycloudflare.com    

In [135]:
import importlib
import run_bot
importlib.reload(run_bot)

print(run_bot.run_bot("GLP-1 trials for type 2 diabetes")["recommendation"])

⏳ Loading pre-built RAG index...
✅ RAG Index Ready: 18063 vectors loaded.


2025-11-27 04:08:01.536 200 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1987.71ms
2025-11-27 04:08:03.854 200 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2314.66ms
2025-11-27 04:08:05.694 200 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1836.68ms
2025-11-27 04:08:08.268 200 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2569.87ms
2025-11-27 04:08:10.513 200 POST /v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 2241.03ms


Here's a structured summary of the provided diabetes clinical trial information, focusing on healthcare relevance:

1️⃣ **Abstract:** This clinical trial compared the efficacy and safety of intravenous glucagon-like peptide-1 (GLP-1) infusion to a standard intravenous insulin infusion protocol for achieving normoglycemia in hyperglycemic patients with type 2 diabetes. The study aimed to evaluate GLP-1 as an alternative to insulin in this patient population.

2️⃣ **Key Findings:** The trial likely investigated whether GLP-1 infusion was non-inferior or superior to insulin infusion in achieving target blood glucose levels in type 2 diabetic patients experiencing hyperglycemia. The study likely assessed the rate of achieving normoglycemia, the time required to reach target glucose, and any differences in adverse events between the two treatment arms.

3️⃣ **Takeaway:** This study provides clinicians with comparative data on the utility of intravenous GLP-1 versus intravenous insulin for a